Google Collab Link: https://colab.research.google.com/drive/1lm2q9YxlLa2lv-78PMLlokkYoWa5ByEZ?usp=sharing


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
cd drive

[Errno 2] No such file or directory: 'drive'
/content/drive/MyDrive/Colab Notebooks


In [2]:
cd MyDrive

/content/drive/MyDrive


In [18]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()


In [3]:
cd Colab Notebooks


/content/drive/MyDrive/Colab Notebooks


In [20]:
ls

 Assignment2.ipynb            SQL2.ipynb   sqli.csv
'Copy of Assignment2.ipynb'   sqli2.csv    SQLi.ipynb


In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score
from scipy.stats import randint
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [22]:
df=pd.read_csv('sqli.csv', encoding='utf-16')
df.head()

,Sentence,Label
0,NaN,1
1,""" or pg_sleep ( __TIME__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' AND 1 = utl_inaddr.get_host_address ( ( S...,1


In [23]:
df['Sentence'] = df['Sentence'].astype(str)
df['Sentence'] = df['Sentence'].str.lower()
df['Label'] = pd.to_numeric(df['Label'], errors='coerce')
df['Label'] = df['Label'].astype('Int64')

In [24]:
df.rename(columns={'Sentence': 'Query'}, inplace=True)
df.head()

,Query,Label
0,nan,1
1,""" or pg_sleep ( __time__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' and 1 = utl_inaddr.get_host_address ( ( s...,1


In [25]:
Final_df_ = df
Final_df_.head()


,Query,Label
0,nan,1
1,""" or pg_sleep ( __time__ ) --",1
2,create user name identified by pass123 tempora...,1
3,%29,1
4,' and 1 = utl_inaddr.get_host_address ( ( s...,1


In [26]:
class CustomDataset(Dataset):
    def __init__(self, texts, labels, vectorizer):
        self.texts = texts
        self.labels = labels
        self.vectorizer = vectorizer

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        # Convert text to numerical representation
        text_vectorized = self.vectorizer.transform([text]).toarray()
        return torch.tensor(text_vectorized), torch.tensor(label)



texts = Final_df_['Query'].tolist()
labels = Final_df_['Label'].tolist()

# Split data into train and test sets
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Initialize CountVectorizer (you can use other tokenizers/embeddings as needed)
vectorizer = CountVectorizer()
vectorizer.fit(texts_train)

# Create custom datasets
train_dataset = CustomDataset(texts_train, labels_train, vectorizer)
test_dataset = CustomDataset(texts_test, labels_test, vectorizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [27]:
test_loader

In [28]:
from transformers import BertTokenizer

# Load pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

class CustomDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Split data into train and test sets
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Create custom datasets
train_dataset = CustomDataset(texts_train, labels_train, tokenizer)
test_dataset = CustomDataset(texts_test, labels_test, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)


In [29]:
from transformers import DistilBertForSequenceClassification
from torch.optim import AdamW
from tqdm import tqdm
import gc

gc.collect()
torch.cuda.empty_cache()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load model
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
model.train()
num_epochs = 2

for epoch in range(num_epochs):
    epoch_loss = 0.0
    correct_preds = 0
    total_preds = 0

    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', unit='batch'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, predicted = torch.max(logits, dim=1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_accuracy = correct_preds / total_preds
    print(f"Epoch {epoch + 1}/{num_epochs} | Loss: {epoch_loss:.4f} | Accuracy: {epoch_accuracy:.4f}")


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Epoch 1/2: 100%|██████████| 6752/6752 [07:04<00:00, 15.91batch/s]


Epoch 1/2 | Loss: 103.1262 | Accuracy: 0.9966


Epoch 2/2: 100%|██████████| 6752/6752 [06:59<00:00, 16.08batch/s]

Epoch 2/2 | Loss: 49.1621 | Accuracy: 0.9983


In [30]:
# Evaluation
model.eval()
eval_loss = 0.0
correct_preds = 0
total_preds = 0

with torch.no_grad():
    for batch in tqdm(test_loader, desc='Evaluation', unit='batch'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, predicted = torch.max(logits, 1)
        correct_preds += (predicted == labels).sum().item()
        total_preds += labels.size(0)

        eval_loss += loss.item()

eval_accuracy = correct_preds / total_preds
print(f'Evaluation Loss: {eval_loss}, Evaluation Accuracy: {eval_accuracy}')


Evaluation: 100%|██████████| 1689/1689 [00:31<00:00, 53.72batch/s]

Evaluation Loss: 16.407158712670935, Evaluation Accuracy: 0.9979268473271139


In [32]:
model.save_pretrained("sqli_model")
tokenizer.save_pretrained("sqli_model")

('sqli_model/tokenizer_config.json',
 'sqli_model/special_tokens_map.json',
 'sqli_model/vocab.txt',
 'sqli_model/added_tokens.json')

In [33]:
import shutil
shutil.make_archive("sqli_model", 'zip', "sqli_model")

'/content/drive/MyDrive/Colab Notebooks/sqli_model.zip'

In [34]:
from google.colab import files
files.download("sqli_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>